In [6]:
import gspread
from google.oauth2.service_account import Credentials
import datetime
import pandas as pd

In [3]:
credentials_filename = "nodal-wall-416914-5876acf16f70.json"
beo_worksheet_id = 2121169814
kimittud_worksheet_id = 858480548
sheet_id = "1fwhZxPtuP2MSBkTXJRSH-plXGV42mF-HlQPf4KSpFwE"

In [7]:
def initialize_gspread(service_account_file_path: str):
    credentials = Credentials.from_service_account_file(
        service_account_file_path,
        scopes=["https://www.googleapis.com/auth/spreadsheets"],
    )
    return gspread.authorize(credentials)


def google_worksheet_to_df(sheet_id: str, worksheet_id: str | int):
    client = initialize_gspread(credentials_filename)
    sheet = client.open_by_key(sheet_id)
    worksheet = sheet.get_worksheet_by_id(worksheet_id)
    data = worksheet.get_all_values()
    return pd.DataFrame(data[1:], columns=data[0])


def get_beosztas_df():
    return google_worksheet_to_df(sheet_id, beo_worksheet_id)


def get_kimittud_df():
    return google_worksheet_to_df(sheet_id, kimittud_worksheet_id)

In [26]:
beo_df = get_beosztas_df()

In [27]:
beo_df

,Dátum,Esemény / \nMegnevezés,Fizu \n(ha nem hétfő),ÁDÁM,ALEX,BALU,BELLA,BORCSA,BORI,DÁVID,...,SANYI,SÁRI,TAKI,VANDA,VERONKA,BARNUS,Jegyszedő,Kulcs,Aktualizálva\n(Főni),Név
0,Hányszor lenne szívesen/maximum,Jelkulcs:\nj - dolgozott (jm) (főni félkövér)\...,Hétfői bérek:\nJm: r:5000 / h:8500\nPak oda: 2...,3,2,3,2,1,4,,...,,0,4,2,0,,,,,
1,Hány %-on áll?,,,52%,52%,81%,54%,63%,52%,52%,...,47%,46%,52%,47%,68%,,,,,
2,Hányszor volt a félévben dolgozni?,,,4,5,10,3,4,7,6,...,1,1,11,4,0,0,,,,
3,2023.12.04.,JH a Turbinában,-,,,j,,,,,...,,,,,,ov,,,2023.02.01.,Dorina
4,2023.12.09.,Csapatépítő - gyógytornász,13 500 Ft,,,,,,,j,...,,,,,,,,,2023.02.01.,Panka
5,2023.12.11.,JH a Turbinában,-,,,jo,,j,,,...,,,,,,,,,2023.02.01.,Dorina
6,2024.01.08.,JH a Turbinában,-,,jv,,,,,,...,,oj,oj,,,ov,,,2024.01.09.,Dorina
7,2024.01.15.,JH a Turbinában,-,,,,,,,,...,,,oj,jv,,,,Vanda,2024.01.16.,Dorina
8,2024.01.18.,Nix / Váci Greens csapatépítő,13500 Ft + 3000 Ft (koordinátor),,,,,,,,...,,,jk,,,,,,2024.01.31.,Balu
9,2024.01.22.,JH a Turbinában,-,,,oj,,j,,,...,jv,,bj,,o,ov,,Sanyi,Főni: Balu,


In [47]:
def cleanup_beo_df(beo_df: pd.DataFrame, max_days_past: int = 100):
    filtered = beo_df.drop(beo_df.columns[[1, 2, -1, -2, -3, -4, -5]], axis=1)
    filtered.drop([0, 1, 2], inplace=True)

    today = datetime.datetime.today()
    min_date = today - datetime.timedelta(days=max_days_past)

    filtered = filtered.loc[
        (filtered["Dátum"] > min_date.strftime("%Y.%m.%d."))
        & (filtered["Dátum"] < today.strftime("%Y.%m.%d."))
    ]

    beo_df.iloc[:, 1:] = beo_df.iloc[:, 1:].apply(lambda x: x.split('') if type(x) == str else x)


    return filtered


cleanup_beo_df(beo_df)

,Dátum,ÁDÁM,ALEX,BALU,BELLA,BORCSA,BORI,DÁVID,DORINA,DORKA,...,LILLA,MÁRK,NIKI,PANKA,RÉKA,SANYI,SÁRI,TAKI,VANDA,VERONKA
8,2024.01.18.,,,,,,,,,,...,,,,,j,,,jk,,
9,2024.01.22.,,,oj,,j,,,,,...,oj,,,,,jv,,bj,,o
10,2024.01.29.,jv,,,,,j,oj,,,...,oj,,,,,v,,oj,,o
11,2024.02.01.,,,,,,,,,,...,,,,,,,,jk,,
12,2024.02.05.,,j,,,,,,,,...,o,oj,,,,,,oj,jv,o
13,2024.02.15.,,,oj,,b,j,,,,...,oj,bo,,v,jv,,,,,o
14,2024.02.19.,,,oj,,,,,,,...,oj,,,,jv,,,,,o
15,2024.02.26.,,,,j,,,v,,,...,oj,o,,,jv,,,oj,j,
16,2024.03.11.,jv,,oj,,,jv,,,,...,,,b,,,v,,,,o
17,2024.03.12.,,,,,,,ojv,,,...,ov,,,,,,,oj,,
